In [47]:
from typing import Optional
import pandas as pd
import numpy as np
import re

df = pd.read_csv('../data/issuers_v2.csv')
#df.drop(columns=['Unnamed: 0'], inplace=True)

In [68]:
df

,issuerid,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,synonims
0,1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Держава, DERZHAVA, DERZ, , , , , , , ]"
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN,"[Московский кредитный банк, мкб, Credit Bank o..."
2,3,"""Российский акционерный коммерческий дорожный ...",NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,roads Bank,NaN,NaN,NaN,[Российский акционерный коммерческий дорожный ...
3,4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[алроса, alrosa, , , , , , , , ]"
4,5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Авангард, AVANGARD, , , , , , , , ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,270,Henderson,NaN,NaN,HNFG,Хендерсон,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Хендерсон, , , , , , , , , ]"
251,271,Совкомбанк,NaN,NaN,SVCB,Sovcombank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Sovcombank, , , , , , , , , ]"
252,272,ЕвроТранс,NaN,NaN,EUTR,АЗС Трасса,"АЗС ""Трасса""",АЗС «Трасса»,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[АЗС Трасса, АЗС ""Трасса"", АЗС «Трасса», , , ,..."
253,273,Делимобиль,NaN,NaN,DELI,Делимобил,Каршеринг Рус,delimobil,"ПАО ""Каршеринг Руссия""","""Каршеринг Руссия"", ПАО",NaN,NaN,NaN,NaN,NaN,"[Делимобил, Каршеринг Рус, delimobil, ПАО ""Кар..."


In [49]:
df.columns

Index(['issuerid', 'EMITENT_FULL_NAME', 'VeryOddCompany', 'BGTicker',
       'BGTicker.1', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')

In [71]:
df['synonims'] = df[['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14']].fillna('').values.tolist()
df['synonims'] = df['synonims'].apply(np.unique).apply(lambda x: [i for i in x if i != ''])

In [72]:
data_companies = df.to_dict('records')

In [73]:
def get_company_name_from_quotes(s: str) -> Optional[str]:
    matches_long = re.findall('".+".+"', s)
    if len(matches_long) > 0:
        return matches_long[0].strip('"').split('"')[1]
    try:
        return re.findall('".+"', s)[0].replace('"', '')
    except Exception as err:
        return None

COMPANY_TYPES = {
    'публичное акционерное общество': 'ПАО',
    'пао': 'ПАО',
    'открытое акционерное общество': 'ОАО',
    'plc': 'PLC', # public limited company
    'публичная компания с ограниченной ответственностью': 'PLC',
    'ltd': 'LTD' # Limited Liability Company
}
def get_company_type(s: str) -> Optional[str]:
    for k,v in COMPANY_TYPES.items():
        if k in s:
            return v
    return None

def get_name_if_not_quotes(s: str) -> str:
    for k in COMPANY_TYPES.keys():
        s = s.replace(k, '')

    s = re.sub('\(\s*\)', '', s)
    s = re.sub('\(эмитент депозитарных расписок.*\)', '', s)
    s = re.sub('\s+', ' ', s).strip()
    return s

In [74]:
get_company_name_from_quotes('Публичное акционерное общество "Угольная компания "Южный Кузбасс"')

'Южный Кузбасс'

In [87]:
new_data = []

for d in data_companies:
    full_name = d['EMITENT_FULL_NAME'].lower()
    company_name = get_company_name_from_quotes(full_name)
    company_type = get_company_type(full_name)
    if company_name is None:
        company_name = get_name_if_not_quotes(full_name)
    new_data.append({
        'issuerid': d['issuerid'],
        'EMITENT_FULL_NAME': d['EMITENT_FULL_NAME'],
        'company_name_lower': company_name,
        'company_type': company_type,
        'BGTickers': [
            i for i in
            [d['BGTicker'], d['BGTicker.1']]
            if isinstance(i, str)
        ],
        'synonims': d['synonims']
    })

In [96]:
import json

with open('../data/issuers_filtered.json', 'wt', encoding='utf-8') as f:
    f.write(json.dumps(new_data, indent=4, ensure_ascii=False))